## Todo list
1. Text Preprocess - done
2. Classify Tweet Function: Trump vs Staff - done
3. Complex features vector -done
4. Build Models
  * LogisticRegression - done
  * SVC - done
  * XGBClassifier
  * FFNN
  * BERT



In [37]:
# PyTprch and Transformers installation and import

# uninstalling dependances to make sure we have only the right versions of them
!pip uninstall -y transformers transformerss
!pip uninstall -y jax jaxlib flax
!pip uninstall -y tokenizers
!pip uninstall -y huggingface-hub
# installing the right version of the transformers library
!pip install --no-cache-dir "transformers[sentencepiece,torch]==4.52.4"
# more installations
! pip install transformers datasets
! pip3 install torch --no-cache-dir
! pip install contractions
! pip install holidays
! pip install optuna
! pip install gdown
! pip install --force-reinstall numpy==1.24.3 scipy==1.11.4 gensim==4.3.1


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: tokenizers 0.21.1
Uninstalling tokenizers-0.21.1:
  Successfully uninstalled tokenizers-0.21.1
Found existing installation: huggingface-hub 0.32.4
Uninstalling huggingface-hub-0.32.4:
  Successfully uninstalled huggingface-hub-0.32.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 294.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 383.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 367.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 247.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 267.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 159.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 269.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883

  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scipy-1.11.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached gensim-4.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached scipy-1.11.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
Using cached gensim-4.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.6 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (83 kB)
  At

In [1]:
# Standard Library
import csv
import re
import requests
import urllib.request
from datetime import datetime
from abc import ABC, abstractmethod
from itertools import combinations

# Third-Party Packages
import contractions
import holidays
import numpy as np
import pandas as pd
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import Word2Vec
import optuna
import inspect
import math
from datasets import Dataset




# Machine Learning
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# --- Transformers (Hugging Face) ---
from scipy.special import softmax
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from xgboost import XGBClassifier
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Import the file
download the csv from google to the colab notebook and open it.

In [2]:
file_id = '1czyLFgxVHKLj9NU2asruGrp2SlL16svb'

# Create a direct download URL to bypass the Drive preview page
url = f'https://drive.google.com/uc?export=download&id={file_id}'

output = 'trump_train.tsv'

# Download and save the file content locally
response = requests.get(url)
with open(output, 'wb') as f:
    f.write(response.content)

In [3]:
file_id = '1qb8IoGnnVmHV2SXx7hg59HA5rtHmGcs1'

# Create a direct download URL to bypass the Drive preview page
url = f'https://drive.google.com/uc?export=download&id={file_id}'

output = 'trump_tweets_test_a.tsv'

# Download and save the file content locally
response = requests.get(url)
with open(output, 'wb') as f:
    f.write(response.content)

## Text Preprocess
1. Pattern-Based Text Normalization using Regular Expressions - Define regular expressions to identify unwanted patterns (e.g., URLs, emails, numbers) and normalize or remove them for cleaner text.
2. Contraction expansion - Expand contractions (e.g., "don't" → "do not").
3. Alphanumeric and Punctuation Filtering - Remove all characters except letters, numbers, and common punctuation to simplify the text.
4. Lowercasing - Convert all characters to lowercase for uniformity (user choise).
5. Lemmatization - Reduce words to their base dictionary form (e.g., "ran" → "run") using tools like WordNetLemmatizer or spaCy.




In [4]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Regex patterns
RE_URL = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
RE_HTML = re.compile(r'<[^<]+?>')
RE_DATE = re.compile(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b|\b\d{2,4}[/-]\d{1,2}[/-]\d{1,2}\b')
RE_DECIMAL = re.compile(r'\b\d+\.\d+%?\b')
RE_NUMBER_PERCENT = re.compile(r'\b\d+%?\b')
RE_SPECIAL_CHAR = re.compile(r'[^0-9a-zA-Z\s?!.,:\'\"//]+')

def Preprocesstext(text, lowercase=False, removestopwords=True):
    """
    Preprocesses a raw input text string by cleaning and normalizing it for NLP tasks.

    The function applies the following steps:
    1. Fixes mojibake characters and normalizes smart quotes.
    2. Expands English contractions (e.g., "don't" → "do not").
    3. Replaces specific patterns with placeholders
    4. Removes unwanted characters except letters, numbers, and basic punctuation.
    5. Splits text into tokens and optionally:
       - Lowercases non-placeholder tokens.
       - Removes stopwords (excluding placeholders).
       - Lemmatizes tokens (excluding placeholders).
    6. Joins the final tokens back into a normalized string.

    Args:
        text (str): The raw input text to preprocess.
        lowercase (bool): If True, converts all non-placeholder tokens to lowercase.
        removestopwords (bool): If True, removes English stopwords (except placeholders).

    Returns:
        str: A cleaned, normalized, and preprocessed text string.
    """

    # Normalize corrupted or special quotes (mojibake cleanup)
    quote_normalization_map = {
        "״": "\"", "“": "\"", "”": "\"",
        "׳": "'", "‘": "'", "’": "'",
        'â\x80\x9c': '"', 'â\x80\x9d': '"', 'â\x80\x99': "'"
    }
    for original_char, normalized_char in quote_normalization_map.items():
        text = text.replace(original_char, normalized_char)

    # Expand contractions (e.g., "it's" → "it is")
    text = contractions.fix(text)

    # Replace structured patterns with placeholder tags
    text = RE_URL.sub('<URL>', text)
    text = RE_HTML.sub('<HTML>', text)
    text = RE_DATE.sub('<DATE>', text)
    text = RE_DECIMAL.sub('<DECIMAL>', text)
    text = RE_NUMBER_PERCENT.sub('<NUMBER>', text)

    # Remove unwanted characters except alphanumerics and basic punctuation
    text = RE_SPECIAL_CHAR.sub('', text)

    # Define protected placeholder tokens
    PLACEHOLDERS = ['<URL>', '<HTML>', '<DATE>', '<DECIMAL>', '<NUMBER>']

    # Tokenize text
    tokens = text.split()

    # Lowercase only non-placeholder tokens
    if lowercase:
        tokens = [t.lower() if t not in PLACEHOLDERS else t for t in tokens]

    # Optionally remove stopwords (keep placeholders)
    if removestopwords:
        tokens = [t for t in tokens if t not in stop_words or t in PLACEHOLDERS]

    # Lemmatize non-placeholder tokens
    tokens = [lemmatizer.lemmatize(t) if t not in PLACEHOLDERS else t for t in tokens]

    # Reconstruct and return cleaned string
    text = ' '.join(tokens)
    return text


def is_valid_timestamp(text, format = '%Y-%m-%d %H:%M:%S'):
    """
    Checks if the input string is in the format 'YYYY-MM-DD HH:MM:SS'.

    Args:
        text (str): The string to validate.

    Returns:
        bool: True if valid timestamp format, False otherwise.
    """
    try:
        datetime.strptime(text, format)
        return True
    except ValueError:
        return False


def average_timestamp(timestamps, format='%Y-%m-%d %H:%M:%S'):
    """
    Calculates the average timestamp from a list of strings, skipping invalid ones.

    Args:
        timestamps (list of str): List of timestamp strings in a consistent format.
        format (str): Expected format of each timestamp. Default: '%Y-%m-%d %H:%M:%S'.

    Returns:
        str: The average timestamp as a string in the same format.
             If no valid timestamps are provided, returns current time.
    """
    # Filter and parse only valid timestamps
    valid_timestamps = [datetime.strptime(ts, format) for ts in timestamps if is_valid_timestamp(ts, format)]

    # Return current time if no valid timestamps
    if not valid_timestamps:
        return datetime.now().strftime(format)

    # Compute average as float (epoch time)
    avg_epoch = sum(map(datetime.timestamp, valid_timestamps)) / len(valid_timestamps)

    # Convert back to datetime and return formatted string
    return datetime.fromtimestamp(avg_epoch).strftime(format)

##Classify Tweet Function: Trump vs Staff

In [5]:
def classify_tweet(device, username, timestamp):
    """
    Classifies a tweet as being from Donald Trump (1), his staff (0), or unclassifiable (None),
    based on historical patterns of device usage and authorship cutoff date.

    Heuristics used:
    - Before April 1, 2017:
        - Tweets from @realDonaldTrump using Android are classified as Trump (1)
        - All other tweets before this date are assumed to be staff (0)
    - On or after April 1, 2017:
        - Device data is no longer reliable for authorship; return None

    Parameters:
        device (str): The device used to send the tweet (e.g., 'Twitter for Android').
        username (str): The username of the account (e.g., 'realDonaldTrump').
        timestamp (str): Timestamp of the tweet in format 'YYYY-MM-DD HH:MM:SS'.

    Returns:
        int: 1 if classified as Trump, 0 if classified as staff.
        None: If timestamp is invalid, missing data, or after the cutoff date.
    """
    try:
        # Normalize constants
        TRUMP_USERNAME = "realDonaldTrump".lower()
        TRUMP_CUTOFF_DATE = datetime.strptime('2017-04-01', '%Y-%m-%d')

        # Parse timestamp
        tweet_date = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')

        # Classification logic
        if username.lower() == TRUMP_USERNAME and 'android' in device.lower() and tweet_date < TRUMP_CUTOFF_DATE:
            return 0  # Likely Trump
        elif tweet_date >= TRUMP_CUTOFF_DATE:
            return None  # Post-cutoff: not classifiable
        else:
            return 1  # Likely staff
    except Exception:
        return None  # Any parsing or data issue

## Complex features vector
This vector is designed to dynamically integrate various types of features into the model input during the optimization process.
It supports combining independent metadata (e.g., device, time), text-based representations like TF-IDF, and semantic embeddings such as Word2Vec, allowing flexible experimentation across different models and configurations.

In [6]:
class AdditionalFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, country="US"):
        """
        Initializes the AdditionalFeatures extractor.

        Args:
            country (str): Country code for holiday detection (e.g., 'US', 'IL').
        """
        self.holiday = holidays.CountryHoliday(country)

    def fit(self, X, y=None):
        return self  # no fitting needed

    def transform(self, X):
        """
        Transforms tweet text and timestamp columns into numerical features.

        Args:
            X (pd.DataFrame or dict-like): Must contain 'tweet text' and 'time stamp' keys.

        Returns:
            np.ndarray: Feature matrix (n_samples x n_features)
        """
        return np.array([
            self.extract_features(text, timestamp)
            for text, timestamp in zip(X['tweet text'], X['time stamp'])
        ])

    def extract_features(self, text, timestamp):
        """
        Extracts features from a single tweet and timestamp.

        Args:
            text (str): The tweet text.
            timestamp (str): Timestamp string in format '%Y-%m-%d %H:%M:%S'.

        Returns:
            list: A fixed-length feature vector.
        """
        try:
            # --- Text features ---
            words = text.split()
            num_words = len(words)
            num_capitalized = sum(1 for w in words if w.isupper())
            num_question_marks = text.count('?')
            num_exclamations = text.count('!')
            avg_word_length = np.mean([len(w) for w in words]) if words else 0
            has_all_caps_word = int(any(w.isupper() and len(w) > 1 for w in words))

            # --- Timestamp features ---
            dt = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
            is_holiday = int(dt.date() in self.holiday)
            hour = dt.hour
            day_of_week = dt.weekday()  # Monday = 0
            month = dt.month
            year = dt.year

            return [
                num_words,
                num_capitalized,
                num_question_marks,
                num_exclamations,
                avg_word_length,
                has_all_caps_word,
                is_holiday,
                hour,
                day_of_week,
                month,
                year
            ]

        except Exception:
            # Fallback: return default values
            return [0] * 11

class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, size=100, min_count=1, window=5, workers=4, sg=1):
        """
        Word2Vec-based vectorizer.

        Args:
            size (int): Dimensionality of word vectors.
            min_count (int): Minimum word frequency for training.
            window (int): Context window size.
            workers (int): Number of worker threads.
            sg (int): Skip-gram (1) vs CBOW (0) mode.
        """
        self.size = size
        self.min_count = min_count
        self.window = window
        self.workers = workers
        self.sg = sg
        self.model = None

    def fit(self, X, y=None):
        """
        Trains Word2Vec model on tokenized input texts.

        Args:
            X (list of str): Input texts.

        Returns:
            self
        """
        tokenized = [text.split() for text in X]
        self.model = Word2Vec(
            sentences=tokenized,
            vector_size=self.size,
            window=self.window,
            min_count=self.min_count,
            workers=self.workers,
            sg=self.sg
        )
        return self

    def transform(self, X):
        """
        Converts each text to an averaged Word2Vec embedding.

        Args:
            X (list of str): Input texts.

        Returns:
            np.ndarray: Matrix of shape (n_samples, size)
        """
        vectors = []
        for text in X:
            tokens = self._tokenize(text)
            vecs = [self.model.wv[word] for word in tokens if word in self.model.wv]
            if vecs:
                vectors.append(np.mean(vecs, axis=0))
            else:
                vectors.append(np.zeros(self.size))
        return np.array(vectors)

    def transform(self, X):
        return np.array([self.vectorize(text) for text in X])

    def vectorize(self, text):
        words = text.split()
        word_vecs = [self.model.wv[word] for word in words if word in self.model.wv]
        if len(word_vecs) == 0:
            return np.zeros(self.size)
        return np.mean(word_vecs, axis=0)



class FeatureVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_types=['tfidf', 'word2vec', 'additional'], country='US',
                 word2vec_size=100, tfidf_max_features=5000):
        """
        Combines multiple feature vector types into a single feature matrix.

        Args:
            vector_types (list): Which vectors to use. Options: 'tfidf', 'word2vec', 'additional'.
            country (str): Country code for holiday feature in AdditionalFeatures.
            word2vec_size (int): Dimensionality of Word2Vec vectors.
            tfidf_max_features (int): Max features for TF-IDF vectorizer.
        """
        self.vector_types = vector_types
        self.country = country
        self.word2vec_size = word2vec_size
        self.tfidf_max_features = tfidf_max_features
        self.additional = AdditionalFeatures(country=self.country)
        # Components (lazy initialization)
        if 'tfidf' in self.vector_types:
            self.tfidf = TfidfVectorizer(max_features=self.tfidf_max_features)
        if 'word2vec' in self.vector_types:
            self.word2vec = Word2VecVectorizer(size=self.word2vec_size)
        if 'additional' in self.vector_types:
            self.additional = AdditionalFeatures(country=self.country)

    def fit(self, X, y=None):
        """
        Fits underlying vectorizers as needed.
        """
        if 'tfidf' in self.vector_types:
            self.tfidf.fit(X['cleaned text'])
        if 'word2vec' in self.vector_types:
            self.word2vec.fit(X['cleaned text'])
        if 'additional' in self.vector_types:
            self.additional.fit(X)
        return self

    def transform(self, X):
        """
        Transforms input data into a combined feature matrix.
        """
        features = []

        if 'tfidf' in self.vector_types:
            tfidf_vec = self.tfidf.transform(X['cleaned text']).toarray()
            features.append(tfidf_vec)

        if 'word2vec' in self.vector_types:
            w2v_vec = self.word2vec.transform(X['cleaned text'])
            features.append(w2v_vec)

        if 'additional' in self.vector_types:
            additional_vec = self.additional.transform(X)
            features.append(additional_vec)

        return np.concatenate(features, axis=1)

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)


**Algorithmic approaches:**



1. [sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
2. [sklearn.svm.SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) (try both linear and nonlinear kernels!)
3. FFNN classifier - You should use  the PyTorch library to build a FFNN classifier (with at least one hidden layer) to achieve the classification. Feel free to experiment with the number of layers ([a simple tutorial for FFNN with PyTorch](https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb)).
4. A fourth classifier of choice (neural or not). You are encouraged to experiment with classifiers that allow combining different types of features (e.g. number of capitalized words, time of tweeting, etc.)
5. A fifth classifier of your choice  (this should be neural -  RNN, or transformer-based) - feel free to experiment.



## Models

In [7]:
class MultiLayerFFNN(nn.Module):
    '''
    Multilayer FFNN
    '''
    def __init__(self, input_dim, hidden_dims, output_dim=2, dropout=0.5):
        super(MultiLayerFFNN, self).__init__()
        layers = []
        current_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(current_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            current_dim = hidden_dim

        layers.append(nn.Linear(current_dim, output_dim))
        layers.append(nn.LogSoftmax(dim=1))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def predict(self, input_data):
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation
            outputs = self(input_data)
            _, predicted = torch.max(outputs, 1)
        return predicted


class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_dims=None, output_dim=2, dropout=0.3, learning_rate=0.001, print=False):
        self.hidden_dims = hidden_dims if hidden_dims is not None else [50, 50]
        self.output_dim = output_dim
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.num_epochs = len(self.hidden_dims) * 50
        self.model = None
        self.criterion = nn.NLLLoss()
        self.optimizer = None
        self.print = print

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        if 'hidden_dims' in params:
            self.num_epochs = len(self.hidden_dims) * 50
        return self

    def fit(self, X, y):
        input_dim = X.shape[1]
        self.model = MultiLayerFFNN(input_dim, self.hidden_dims, self.output_dim, self.dropout)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.to_numpy(), dtype=torch.long)

        dataset = TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

        for epoch in range(self.num_epochs):
            self.model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
            self.print and print(f"Epoch [{epoch+1}/{self.num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

        return self

    def predict(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        predictions = self.model.predict(X_tensor)
        return predictions.numpy()



class BertHFModel:
    def __init__(
        self,
        text_col: str = "cleaned text",
        model_name: str = "prajjwal1/bert-mini",
        max_len: int = 64,
        epochs: int = 4,
        batch_size: int = 8,
        lr: float = 3e-5,
        warmup_ratio: float = 0.1,
        weight_decay: float = 0.01,
        unfreeze_last_n: int = 2,
        fp16: bool | None = None,
        output_dir: str = "fast_bert_plus_ckpt",
        verbose: bool = True,
    ):
        self.text_col = text_col
        self.max_len = max_len
        self.epochs = epochs
        self.batch = batch_size
        self.lr = lr
        self.warmup = warmup_ratio
        self.wd = weight_decay
        self.out_dir = output_dir
        self.verbose = verbose
        self.fp16 = torch.cuda.is_available() if fp16 is None else fp16
        self.unfreeze = unfreeze_last_n

        self.tok = BertTokenizerFast.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=2
        )

        # Unfreeze last N encoder layers + classifier + LayerNorm
        enc_layers = list(self.model.bert.encoder.layer)
        for name, p in self.model.named_parameters():
            p.requires_grad = (
                name.startswith("classifier")
                or any(name.startswith(f"bert.encoder.layer.{i}") for i in range(len(enc_layers) - self.unfreeze, len(enc_layers)))
                or "LayerNorm" in name
            )

        self.trainer: Trainer | None = None

    @staticmethod
    def _make_args(out_dir: str, **kw):
        import inspect
        TA = TrainingArguments
        sig = inspect.signature(TA.__init__).parameters
        filt = {k: v for k, v in kw.items() if k in sig}
        if "evaluation_strategy" not in sig or "save_strategy" not in sig:
            filt.pop("load_best_model_at_end", None)
            filt.pop("metric_for_best_model", None)
        return TA(out_dir, **filt)

    def train(self, df: pd.DataFrame, label_col="author"):
        ds = Dataset.from_pandas(df[[self.text_col, label_col]])
        tok_ds = (
            ds.map(
                lambda e: self.tok(
                    e[self.text_col],
                    padding="max_length",
                    truncation=True,
                    max_length=self.max_len,
                ),
                batched=True,
            )
            .rename_column(label_col, "labels")
            .remove_columns([self.text_col])
        )
        tok_ds.set_format("torch")

        y = np.array(tok_ds["labels"])
        idx_tr, idx_val = train_test_split(np.arange(len(y)), test_size=0.1, stratify=y, random_state=42)
        train_ds, val_ds = tok_ds.select(idx_tr), tok_ds.select(idx_val)

        args = self._make_args(
            self.out_dir,
            per_device_train_batch_size=self.batch,
            per_device_eval_batch_size=self.batch,
            num_train_epochs=self.epochs,
            learning_rate=self.lr,
            warmup_ratio=self.warmup,
            weight_decay=self.wd,
            fp16=self.fp16,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            logging_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            report_to="none",
        )

        def metrics(pred):
            logit, label = pred
            p = logit.argmax(-1)
            acc = accuracy_score(label, p)
            prec, rec, f1, _ = precision_recall_fscore_support(label, p, average="binary")
            return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

        self.trainer = Trainer(
            model=self.model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=val_ds,
            tokenizer=self.tok,
            data_collator=DataCollatorWithPadding(self.tok),
            compute_metrics=metrics,
        )

        if self.verbose:
            print("⚡ Fine-tuning BERT-mini (last-2 layers) …")
        self.trainer.train()

        best_acc = getattr(self.trainer.state, "best_metric", None)
        best_step = getattr(self.trainer.state, "best_step", None)
        if best_acc is not None and best_step is not None:
            steps_per_epoch = math.ceil(len(train_ds) / self.batch)
            best_epoch = int(round(best_step / steps_per_epoch))
            best_params = {
                "lr": self.lr,
                "unfreeze_last": self.unfreeze,
                "max_len": self.max_len,
                "batch_size": self.batch,
            }
            print(f"Best Epoch #{best_epoch}: Accuracy={best_acc:.4f} | Params={best_params}")

    def predict(self, df: pd.DataFrame):
        if self.trainer is None:
            raise ValueError("Call train() first.")
        ds = Dataset.from_pandas(df[[self.text_col]])
        ds = ds.map(
            lambda e: self.tok(
                e[self.text_col],
                padding="max_length",
                truncation=True,
                max_length=self.max_len,
            ),
            batched=True,
        ).remove_columns([self.text_col])
        ds.set_format("torch")
        logits = self.trainer.predict(ds).predictions
        return logits.argmax(-1)



##API

Your notebook should support the following functions:

In [8]:
hyperparameters = {
    1: {
        'model_name': 'LogisticRegression',
        'vector': ['tfidf'],
        'best_hyperparams': {
           'C': 9.864948922299297,
        }


    },
    2: {
        'model_name': 'SVC',
        'vector': ['tfidf', 'additional'],
        'best_hyperparams': {
            'kernel': 'rbf',
            'C': 4.31289165277942,
            'gamma': 0.030396135757067042
        }
    },
    3: {
        'model_name': 'XGBClassifier',
        'vector': ['tfidf', 'additional'],
        'best_hyperparams': {
            'n_estimators': 477,
            'max_depth': 9,
            'learning_rate': 0.016548545657454578,
            'subsample': 0.8444703426347393,
            'colsample_bytree': 0.6238505376857482,
            'reg_lambda': 0.4156047528735853
        }
    },
    4: {
        'model_name': 'FFNN',
        'vector': ['tfidf'],
        'best_hyperparams': {
            'hidden': [50],
        }
    },
    5: {
        'model_name': 'BERT',
        'vector': None,
        'best_hyperparams': None


    }
}

models = {
    1: LogisticRegression(max_iter=1000, **hyperparameters[1]['best_hyperparams']),
    2: SVC(**hyperparameters[2]['best_hyperparams']),
    3: XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', **hyperparameters[3]['best_hyperparams']),
    4: PyTorchClassifier(hidden_dims=hyperparameters[4]['best_hyperparams']['hidden']),
    5: BertHFModel()
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def training_pipeline(alg, train_fn='trump_train.tsv', test=True, train=False):
    """
      Trains a model (1=LR, 2=SVM, 3=FFNN, 4=PyTorch FFNN, 5=BERT) with preprocessing and evaluation.

      Args:
          alg (int): Algorithm ID.
          train_fn (str): Path to training file.
          test (bool): Whether to split for evaluation.
          train (bool): Whether to train or use fixed hyperparameters.

      Returns:
          Trained pipeline or BERT model.
    """

    column_names = ['tweet id', 'user handle', 'tweet text', 'time stamp', 'device']
    df = pd.read_csv(train_fn, sep='\t', names=column_names, header=None, quoting=csv.QUOTE_NONE)

    print("🔄 Preprocessing...")
    df['cleaned text'] = df['tweet text'].apply(Preprocesstext)
    avg_ts = average_timestamp(df["time stamp"])
    df["time stamp"] = df["time stamp"].apply(lambda ts: ts if is_valid_timestamp(ts) else avg_ts)
    df["author"] = df.apply(lambda row: classify_tweet(row["device"], row["user handle"], row["time stamp"]), axis=1)
    df.dropna(subset=["author"], inplace=True)
    df["author"] = df["author"].astype(int)

    if test:
        df_train, df_test = train_test_split(df, test_size=0.2, stratify=df["author"], random_state=42)
    else:
        df_train, df_test = df, None

    if alg < 5:
        if train:
            print("🧪 Training with all vectorizer combinations and cross-validation...")

            base_vectors = ['tfidf', 'word2vec']
            feature_combinations = [
                [bv] for bv in base_vectors
            ] + [
                [bv, 'additional'] for bv in base_vectors
            ]

            best_score = 0
            best_vector_types = None
            best_model = None
            trial = 1

            for vt in feature_combinations:
                print(f"\n🔁 Trial #{trial}")
                print(f"🔢 Vector Types: {vt}")
                model = models[alg]
                print(f"🧪 Hyperparameters: {model.get_params()}")

                pipeline = Pipeline([
                    ('features', FeatureVectorizer(vector_types=vt, country='US')),
                    ('classifier', model)
                ])

                scores = cross_val_score(pipeline, df_train.drop(columns=['author']), df_train['author'], cv=3, scoring='f1')
                avg_score = scores.mean()
                print(f"📉 F1 scores per fold: {scores}")
                print(f"📈 Avg F1-score: {avg_score:.4f}")

                if avg_score > best_score:
                    best_score = avg_score
                    best_vector_types = vt
                    best_model = model

                trial += 1

            print("\n🏁 Final Best Configuration")
            print(f"✅ Vector Types: {best_vector_types}")
            print(f"✅ F1 Score: {best_score:.4f}")
            print(f"✅ Hyperparameters: {best_model.get_params()}")

            pipeline = Pipeline([
                ('features', FeatureVectorizer(vector_types=best_vector_types, country='US')),
                ('classifier', best_model)
            ])
            pipeline.fit(df_train.drop(columns=['author']), df_train['author'])

        else:
            vector_types = hyperparameters[alg]['vector']
            print(f"📦 Loaded vector types from config: {vector_types}")

            model = models[alg]
            pipeline = Pipeline([
                ('features', FeatureVectorizer(vector_types=vector_types, country='US')),
                ('classifier', model)
            ])
            pipeline.fit(df_train.drop(columns=['author']), df_train['author'])

    else:
        model = BertHFModel()
        model.train(df_train, label_col="author")
        pipeline = model

    if test:
        X_test = df_test.drop(columns=['author'])
        y_test = df_test['author']
        y_pred = pipeline.predict(X_test)

        print("\n🧪 Evaluation on Test Set")
        print(classification_report(y_test, y_pred))
        print(f"Accuracy : {accuracy_score(y_test, y_pred):.2f}")
        print(f"F1 Score : {f1_score(y_test, y_pred):.2f}")
        print(f"Recall   : {recall_score(y_test, y_pred):.2f}")
        print(f"Precision: {precision_score(y_test, y_pred):.2f}")

    return pipeline


In [23]:
def retrain_best_model():
  """ Retrains and returns the best performing model for the specified task. The
      function uses the hard coded settings you have found to work best for each
      of the tasks.

      Args:

  """
  # Create a pipeline with the custom feature vectorizer and the classifier
  m = training_pipeline(3, test = False)

  return m

In [24]:
def predict(pipeline, fn):
    """ Returns a list of 0s and 1s, corresponding to the lines in the specified file.

    Args:
        pipeline: the trained model and process to be used.
        fn: the full path to a file in the same format as the test set we have provided.
    """
    print('🔄 Pre-Process Data')
    # Define the column names
    column_names = ['user handle', 'tweet text', 'time stamp']

    # Load the TSV file into a DataFrame with specified column names
    df = pd.read_csv(fn, sep='\t', names=column_names, header=None, quoting=csv.QUOTE_NONE)

    # Process the Text
    df['cleaned text'] = df['tweet text'].apply(Preprocesstext)
    avg_timestamp = average_timestamp(df['time stamp'])
    df["time stamp"] = df["time stamp"].apply(lambda ts: ts if is_valid_timestamp(ts) else avg_timestamp)

    print('📊 Predicting...')
    if isinstance(pipeline, Pipeline):
        predictions = pipeline.predict(df)
    elif hasattr(pipeline, "predict"):
        predictions = pipeline.predict(df)  # this is your BertHFModel
    else:
        raise ValueError("Unsupported model type for prediction")

    return predictions.tolist()


In [ ]:
def who_am_i():  # this is not a class method
    """Returns a list of dictionaries, each dictionary with your name, id number and email. keys=['name', 'id','email']
        (If you are submitting solo, the list should contain only one dictionary. If you submit as a team, the list should
        contain a dictionary for each team member.)
        Make sure you return your own info!
    """
    return [{'name': 'Nir Rahav', 'id': '316275437', 'email': 'rahavn@post.bgu.ac.il'}, {'name': 'Yael Berkovich', 'id': '324879501', 'email': 'berkova@post.bgu.ac.il'}]

## Local testing

In [10]:
train_fn = '/content/trump_train.tsv'
test_fn = '/content/trump_tweets_test_a.tsv'


In [15]:
model = training_pipeline(1, train_fn, test = True)

🔄 Preprocessing...
📦 Loaded vector types from config: ['tfidf']

🧪 Evaluation on Test Set
              precision    recall  f1-score   support

           0       0.86      0.94      0.90       451
           1       0.87      0.73      0.79       255

    accuracy                           0.86       706
   macro avg       0.86      0.83      0.84       706
weighted avg       0.86      0.86      0.86       706

Accuracy : 0.86
F1 Score : 0.79
Recall   : 0.73
Precision: 0.87


In [18]:
model = training_pipeline(2, train_fn, test = True, train = True)

🔄 Preprocessing...
🧪 Training with all vectorizer combinations and cross-validation...

🔁 Trial #1
🔢 Vector Types: ['tfidf']
🧪 Hyperparameters: {'C': 4.31289165277942, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.030396135757067042, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
📉 F1 scores per fold: [0.66159696 0.68805704 0.69776119]
📈 Avg F1-score: 0.6825

🔁 Trial #2
🔢 Vector Types: ['word2vec']
🧪 Hyperparameters: {'C': 4.31289165277942, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.030396135757067042, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
📉 F1 scores per fold: [0.07282913 0.0509915  0.        ]
📈 Avg F1-score: 0.0413

🔁 Trial #3
🔢 Vector Typ

In [19]:
model = training_pipeline(3, train_fn, test = True, train = True)

🔄 Preprocessing...
🧪 Training with all vectorizer combinations and cross-validation...

🔁 Trial #1
🔢 Vector Types: ['tfidf']
🧪 Hyperparameters: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6238505376857482, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.016548545657454578, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 477, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': 0.4156047528735853, 'sampling_method': None, 'scale_pos_weight': None

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:34:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:34:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:35:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📉 F1 scores per fold: [0.76872964 0.76507937 0.77408638]
📈 Avg F1-score: 0.7693

🔁 Trial #2
🔢 Vector Types: ['word2vec']
🧪 Hyperparameters: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6238505376857482, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.016548545657454578, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 477, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': 0.4156047528735853, 'sampling_method': None, 'scale_pos_weight': None, 's

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:35:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:35:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:35:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📉 F1 scores per fold: [0.70546737 0.74662162 0.7137931 ]
📈 Avg F1-score: 0.7220

🔁 Trial #3
🔢 Vector Types: ['tfidf', 'additional']
🧪 Hyperparameters: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6238505376857482, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.016548545657454578, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 477, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': 0.4156047528735853, 'sampling_method': None, 'scale_pos_weight

...

📉 F1 scores per fold: [0.83256528 0.82098765 0.84745763]
📈 Avg F1-score: 0.8337

🔁 Trial #4
🔢 Vector Types: ['word2vec', 'additional']
🧪 Hyperparameters: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6238505376857482, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.016548545657454578, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 477, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': 0.4156047528735853, 'sampling_method': None, 'scale_pos_wei

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:36:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:36:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:36:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📉 F1 scores per fold: [0.76948052 0.81037277 0.75693312]
📈 Avg F1-score: 0.7789

🏁 Final Best Configuration
✅ Vector Types: ['tfidf', 'additional']
✅ F1 Score: 0.8337
✅ Hyperparameters: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6238505376857482, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.016548545657454578, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 477, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': 0.4156047528735853, 'sampli

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:37:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🧪 Evaluation on Test Set
              precision    recall  f1-score   support

           0       0.89      0.94      0.91       451
           1       0.88      0.78      0.83       255

    accuracy                           0.89       706
   macro avg       0.89      0.86      0.87       706
weighted avg       0.89      0.89      0.88       706

Accuracy : 0.89
F1 Score : 0.83
Recall   : 0.78
Precision: 0.88


In [20]:
model = training_pipeline(4, train_fn, test = True, train = True)

🔄 Preprocessing...
🧪 Training with all vectorizer combinations and cross-validation...

🔁 Trial #1
🔢 Vector Types: ['tfidf']
🧪 Hyperparameters: {'dropout': 0.3, 'hidden_dims': [50], 'learning_rate': 0.001, 'output_dim': 2, 'print': False}


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


📉 F1 scores per fold: [0.71007752 0.70679012 0.7195122 ]
📈 Avg F1-score: 0.7121

🔁 Trial #2
🔢 Vector Types: ['word2vec']
🧪 Hyperparameters: {'dropout': 0.3, 'hidden_dims': [50], 'learning_rate': 0.001, 'output_dim': 2, 'print': False}


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


📉 F1 scores per fold: [0.6996337  0.67030965 0.72277228]
📈 Avg F1-score: 0.6976

🔁 Trial #3
🔢 Vector Types: ['tfidf', 'additional']
🧪 Hyperparameters: {'dropout': 0.3, 'hidden_dims': [50], 'learning_rate': 0.001, 'output_dim': 2, 'print': False}


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


📉 F1 scores per fold: [0.59491194 0.         0.58027079]
📈 Avg F1-score: 0.3917

🔁 Trial #4
🔢 Vector Types: ['word2vec', 'additional']
🧪 Hyperparameters: {'dropout': 0.3, 'hidden_dims': [50], 'learning_rate': 0.001, 'output_dim': 2, 'print': False}


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


📉 F1 scores per fold: [0.39277652 0.57644991 0.61056106]
📈 Avg F1-score: 0.5266

🏁 Final Best Configuration
✅ Vector Types: ['tfidf']
✅ F1 Score: 0.7121
✅ Hyperparameters: {'dropout': 0.3, 'hidden_dims': [50], 'learning_rate': 0.001, 'output_dim': 2, 'print': False}

🧪 Evaluation on Test Set
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       451
           1       0.75      0.69      0.72       255

    accuracy                           0.80       706
   macro avg       0.79      0.78      0.78       706
weighted avg       0.80      0.80      0.80       706

Accuracy : 0.80
F1 Score : 0.72
Recall   : 0.69
Precision: 0.75


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [21]:
model = training_pipeline(5, train_fn, test = True, train = True)

🔄 Preprocessing...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2821 [00:00<?, ? examples/s]

<ipython-input-7-4e66316920a8>:183: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(


⚡ Fine-tuning BERT-mini (last-2 layers) …


...

Step,Training Loss
318,0.532900
636,0.330700
954,0.300400
1272,0.288400


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or

Map:   0%|          | 0/706 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)



🧪 Evaluation on Test Set
              precision    recall  f1-score   support

           0       0.86      0.98      0.91       451
           1       0.94      0.71      0.81       255

    accuracy                           0.88       706
   macro avg       0.90      0.84      0.86       706
weighted avg       0.89      0.88      0.88       706

Accuracy : 0.88
F1 Score : 0.81
Recall   : 0.71
Precision: 0.94


In [25]:
model = retrain_best_model()

🔄 Preprocessing...
📦 Loaded vector types from config: ['tfidf', 'additional']


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:42:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [29]:
# Get predictions
res = predict(model, test_fn)

# Convert predictions to space-separated string
pred_string = ' '.join([str(pred) for pred in res])
print(pred_string)

🔄 Pre-Process Data
📊 Predicting...
1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0


In [31]:
with open("predictions.txt", "w") as f:
    f.write(pred_string)

print("✅ Predictions exported to predictions.txt")

✅ Predictions exported to predictions.txt
